Version 1

In [1]:
import cv2
import numpy as np
import os
import tkinter as tk
from tkinter import filedialog, ttk

class LucasKanade_OpticalFlow:
    def __init__(self, video_path=None):
        self.video_path = video_path
        self.cap = cv2.VideoCapture(self.video_path) if self.video_path else None
        self.output_folder = None
        self.frame_count = 0

    def create_output_folder(self):
        folder_number = 1

        while True:
            current_folder = f'optical_flow_images_{folder_number:03d}'
            if not os.path.exists(current_folder):
                os.makedirs(current_folder)
                self.output_folder = current_folder
                break
            else:
                folder_number += 1
    #Use Shi-Tomasi Corner Detection
    def detect_corners(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners = cv2.goodFeaturesToTrack(gray, maxCorners=3000, qualityLevel=0.001, minDistance=0.1)
        return corners, gray

    def calculate_optical_flow(self, prev_gray, gray, prev_pts):
        new_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_pts, None, winSize=(25, 25), maxLevel=3,
                                                        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))
        flow_image = np.zeros_like(prev_gray)
        for new, old in zip(new_pts, prev_pts):
            a, b = new.ravel()
            c, d = old.ravel()
            velocity_x = a - c
            velocity_y = b - d
            intensity = int(np.sqrt(velocity_x**2 + velocity_y**2) * 255 / 5)
            color = (intensity, 0, 0)
            flow_image = cv2.line(flow_image, (int(c), int(d)), (int(a), int(b)), color, 2)
        return flow_image

    def track_features(self):
        self.get_video_path
        if not self.cap:
            return

        ret, frame = self.cap.read()
        prev_corners, prev_gray = self.detect_corners(frame)

        while ret:
            ret, frame = self.cap.read()
            if not ret:
                break

            new_corners, gray = self.detect_corners(frame)
            flow_image = self.calculate_optical_flow(prev_gray, gray, prev_corners)

            prev_gray = gray
            prev_corners = new_corners.reshape(-1, 1, 2)

            cv2.imshow('Optical Flow', flow_image)
            k = cv2.waitKey(30) & 0xff
            if k == 27 or k == ord('q'):  # 'q' key to quit
                break

        self.cap.release()
        cv2.destroyAllWindows()

    def get_video_path(self):
        file_path = filedialog.askopenfilename(title="Select Video File", filetypes=[("Video files", "*.mp4;*.avi")])
        if file_path:
            self.video_path = file_path
            self.cap = cv2.VideoCapture(self.video_path)

    def get_optical_images(self, progress_var):
        self.get_video_path()
        if not self.cap:
            return
        #adjust tkinter window to center
        progress_window = tk.Toplevel()  
        progress_window.title("Optical Flow Conversion Progress")
        screen_width = progress_window.winfo_screenwidth()
        screen_height = progress_window.winfo_screenheight()
        x_coordinate = (screen_width - 500) // 2
        y_coordinate = (screen_height - 500) // 2
        progress_window.geometry(f"500x200+{x_coordinate}+{y_coordinate}")

        progress_label = tk.Label(progress_window, text="Optical Flow Conversion Progress:")
        progress_label.pack(pady=10)

        progress_bar = ttk.Progressbar(progress_window, variable=progress_var, length=200, mode='determinate')
        progress_bar.pack(pady=10)

        if not self.cap:
            return

        self.create_output_folder()

        total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))

        ret, frame = self.cap.read()
        prev_corners, prev_gray = self.detect_corners(frame)

        while ret:
            ret, frame = self.cap.read()
            if not ret:
                break

            new_corners, gray = self.detect_corners(frame)
            flow_image = self.calculate_optical_flow(prev_gray, gray, prev_corners)

            cv2.imwrite(os.path.join(self.output_folder, f'optical_flow_{self.frame_count + 1:04d}.png'), flow_image)

            progress_var.set(int((self.frame_count / total_frames) * 100))  # Update progress

            k = cv2.waitKey(30) & 0xff
            if k == 27 or k == ord('q'):  # 'q' key to quit
                break

            self.frame_count += 1
            prev_gray = gray
            prev_corners = new_corners.reshape(-1, 1, 2)

            progress_window.update()

        self.frame_count = 0
        self.cap.release()
        cv2.destroyAllWindows()

        progress_window.destroy()

    def show_optical_flow_video(self):
        self.get_video_path()
        self.track_features()

    def live_optical_flow_tracking(self):
        self.cap = cv2.VideoCapture(0)  # Use camera feed
        self.track_features()

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Lucas-Kanade Optical Flow")

    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    x_coordinate = (screen_width - 500) // 2
    y_coordinate = (screen_height - 500) // 2
    root.geometry(f"500x200+{x_coordinate}+{y_coordinate}")
    feature_tracker = LucasKanade_OpticalFlow()

    def get_optical_images():
        progress_var = tk.IntVar()
        progress_var.set(0)
        feature_tracker.get_optical_images(progress_var)

    def show_optical_flow_video():
        feature_tracker.show_optical_flow_video()

    def live_optical_flow_tracking():
        feature_tracker.live_optical_flow_tracking()

    def exit_application():
        root.destroy()

    # Buttons
    btn_get_optical_images = tk.Button(root, text="Get Optical Images", command=get_optical_images)
    btn_get_optical_images.pack(pady=10)

    btn_show_optical_flow = tk.Button(root, text="Show Optical Flow Video", command=show_optical_flow_video)
    btn_show_optical_flow.pack(pady=10)

    btn_live_tracking = tk.Button(root, text="Live Optical Flow Tracking", command=live_optical_flow_tracking)
    btn_live_tracking.pack(pady=10)

    btn_exit = tk.Button(root, text="Exit", command=exit_application)
    btn_exit.pack(pady=10)

    root.mainloop()


In [5]:
import cv2
import numpy as np
import os
import tkinter as tk
from tkinter import filedialog, ttk

class OpticalFlows:
    def __init__(self, video_path=None):
        self.video_path = video_path
        self.cap = cv2.VideoCapture(self.video_path) if self.video_path else None
        self.output_folder = None
        self.frame_count = 0

    def create_output_folder(self):
        folder_number = 1

        while True:
            current_folder = f'optical_flow_images_{folder_number:03d}'
            if not os.path.exists(current_folder):
                os.makedirs(current_folder)
                self.output_folder = current_folder
                break
            else:
                folder_number += 1
    
    def get_video_path(self):
        file_path = filedialog.askopenfilename(title="Select Video File", filetypes=[("Video files", "*.mp4;*.avi")])
        if file_path:
            self.video_path = file_path
            self.cap = cv2.VideoCapture(self.video_path)

    #Functions for Calculating Each Optical Flow Methods
    def calculate_optical_flow_LK(self, prev_gray, gray, prev_pts):
        new_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_pts, None, winSize=(25, 25), maxLevel=3,
                                                        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))
        flow_image = np.zeros_like(prev_gray)
        for new, old in zip(new_pts, prev_pts):
            a, b = new.ravel()
            c, d = old.ravel()
            velocity_x = a - c
            velocity_y = b - d
            intensity = int(np.sqrt(velocity_x**2 + velocity_y**2) * 255 / 5)
            color = (intensity, 0, 0)
            flow_image = cv2.line(flow_image, (int(c), int(d)), (int(a), int(b)), color, 2)
        return flow_image

    def calculate_optical_flow_FB(self, prev_gray, gray):
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

        hsv = np.zeros_like(self.first_frame)
        hsv[..., 1] = 255
        hsv[..., 0] = (angle * 180 / np.pi / 2).astype(np.uint8)
        hsv[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

        flow_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

        return flow_image
    
    #This one uses Shi-Tomasi for Lucas-Kanade
    def detect_corners(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners = cv2.goodFeaturesToTrack(gray, maxCorners=3000, qualityLevel=0.001, minDistance=0.1)
        return corners, gray
    

    #Tracking functions for Lucas-Kanade and Farneback algorithm
    def track_features_LK(self):
        self.get_video_path
        if not self.cap:
            return

        ret, frame = self.cap.read()
        prev_corners, prev_gray = self.detect_corners(frame)

        while ret:
            ret, frame = self.cap.read()
            if not ret:
                break

            new_corners, gray = self.detect_corners(frame)
            flow_image = self.calculate_optical_flow_LK(prev_gray, gray, prev_corners)

            prev_gray = gray
            prev_corners = new_corners.reshape(-1, 1, 2)

            cv2.imshow('Optical Flow', flow_image)
            k = cv2.waitKey(30) & 0xff
            if k == 27 or k == ord('q'):  # 'q' key to quit
                break

        self.cap.release()
        cv2.destroyAllWindows()

    def track_features_FB(self):
        while True:
            ret, frame = self.cap.read()
            if not ret:
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            flow_image = self.calculate_optical_flow_LK(self.prev_gray, gray)
            cv2.imshow('Optical Flow', flow_image)

            k = cv2.waitKey(30) & 0xff
            if k == 27 or k == ord('q'):  # 'q' key to quit
                break

            self.prev_gray = gray
            self.frame_count += 1

        self.cap.release()
        cv2.destroyAllWindows()


    #Functions for getting Optical Images
    def get_optical_images_LK(self, progress_var):
        self.get_video_path()
        if not self.cap:
            return

        progress_window = tk.Toplevel()  
        progress_window.title("Optical Flow Conversion Progress")
        screen_width = progress_window.winfo_screenwidth()
        screen_height = progress_window.winfo_screenheight()
        x_coordinate = (screen_width - 500) // 2
        y_coordinate = (screen_height - 500) // 2
        progress_window.geometry(f"500x200+{x_coordinate}+{y_coordinate}")

        progress_label = tk.Label(progress_window, text="Optical Flow Conversion Progress:")
        progress_label.pack(pady=10)

        progress_bar = ttk.Progressbar(progress_window, variable=progress_var, length=200, mode='determinate')
        progress_bar.pack(pady=10)

        if not self.cap:
            return

        self.create_output_folder()

        total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))

        ret, frame = self.cap.read()
        prev_corners, prev_gray = self.detect_corners(frame)

        while ret:
            ret, frame = self.cap.read()
            if not ret:
                break

            new_corners, gray = self.detect_corners(frame)
            flow_image = self.calculate_optical_flow_LK(prev_gray, gray, prev_corners)

            cv2.imwrite(os.path.join(self.output_folder, f'optical_flow_{self.frame_count + 1:04d}.png'), flow_image)

            progress_var.set(int((self.frame_count / total_frames) * 100))  # Update progress

            k = cv2.waitKey(30) & 0xff
            if k == 27 or k == ord('q'):  # 'q' key to quit
                break

            self.frame_count += 1
            prev_gray = gray
            prev_corners = new_corners.reshape(-1, 1, 2)

            progress_window.update()

        self.frame_count = 0
        self.cap.release()
        cv2.destroyAllWindows()

        progress_window.destroy()

    def get_optical_images_FB(self,progress_var):
        self.get_video_path()
        if not self.cap:
            return

        progress_window = tk.Toplevel()  
        progress_window.title("Optical Flow Conversion Progress")
        screen_width = progress_window.winfo_screenwidth()
        screen_height = progress_window.winfo_screenheight()
        x_coordinate = (screen_width - 500) // 2
        y_coordinate = (screen_height - 500) // 2
        progress_window.geometry(f"500x200+{x_coordinate}+{y_coordinate}")

        progress_label = tk.Label(progress_window, text="Optical Flow Conversion Progress:")
        progress_label.pack(pady=10)

        progress_bar = ttk.Progressbar(progress_window, variable=progress_var, length=200, mode='determinate')
        progress_bar.pack(pady=10)

        if not self.cap:
            return

        self.create_output_folder()

        total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        while True:
            ret, frame = self.cap.read()
            if not ret:
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            flow_image = self.calculate_optical_flow_FB(self.prev_gray, gray)
            cv2.imshow('Optical Flow', flow_image)

            image_filename = f'optical_flow_{self.frame_count:04d}.png'
            cv2.imwrite(os.path.join(self.output_folder, image_filename), flow_image)

            progress_var.set(int((self.frame_count / total_frames) * 100))  # Update progress

            k = cv2.waitKey(30) & 0xff
            if k == 27 or k == ord('q'):  # 'q' key to quit
                break

            self.prev_gray = gray
            self.frame_count += 1

        self.cap.release()
        cv2.destroyAllWindows()


    #Functions for showing Optical Flow
    def show_optical_flow_video_LK(self):
        self.get_video_path()
        self.track_features_LK()

    def show_optical_flow_video_FB(self):
        self.get_video_path()
        self.track_features_FB()

    def live_optical_flow_tracking(self):
        self.cap = cv2.VideoCapture(0)  # Use camera feed
        self.track_features()

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Lucas-Kanade Optical Flow")

    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    x_coordinate = (screen_width - 500) // 2
    y_coordinate = (screen_height - 500) // 2
    root.geometry(f"500x200+{x_coordinate}+{y_coordinate}")
    feature_tracker = OpticalFlows()

    def get_optical_images_LK():
        progress_var = tk.IntVar()
        progress_var.set(0)
        feature_tracker.get_optical_images_LK(progress_var)

    def get_optical_images_FB():
        progress_var = tk.IntVar()
        progress_var.set(0)
        feature_tracker.get_optical_images_FB(progress_var)

    def show_optical_flow_video_LK():
        feature_tracker.show_optical_flow_video_LK()

    def show_optical_flow_video_FB():
        feature_tracker.show_optical_flow_video_FB()

    def live_optical_flow_tracking_LK():
        feature_tracker.live_optical_flow_tracking_LK()

    def live_optical_flow_tracking_FB():
        feature_tracker.live_optical_flow_tracking_FB()

    def exit_application():
        root.destroy()

    # Buttons
    btn_get_optical_images = tk.Button(root, text="Get Optical Images", command=get_optical_images_LK)
    btn_get_optical_images.pack(pady=10)

    btn_show_optical_flow = tk.Button(root, text="Show Optical Flow Video", command=show_optical_flow_video_LK)
    btn_show_optical_flow.pack(pady=10)

    btn_live_tracking = tk.Button(root, text="Live Optical Flow Tracking", command=live_optical_flow_tracking_LK)
    btn_live_tracking.pack(pady=10)

    btn_exit = tk.Button(root, text="Exit", command=exit_application)
    btn_exit.pack(pady=10)

    root.mainloop()


In [8]:
import cv2
import numpy as np
import os
import tkinter as tk
from tkinter import filedialog, ttk

class OpticalFlows:
    def __init__(self):
        self.video_path = None
        self.cap = None
        self.output_folder = None
        self.frame_count = 0

    def create_output_folder(self):
        folder_number = 1

        while True:
            current_folder = f'optical_flow_images_{folder_number:03d}'
            if not os.path.exists(current_folder):
                os.makedirs(current_folder)
                self.output_folder = current_folder
                break
            else:
                folder_number += 1
    def get_video_path(self):
        file_path = filedialog.askopenfilename(title="Select Video File", filetypes=[("Video files", "*.mp4;*.avi")])
        if file_path:
            self.video_path = file_path
            self.cap = cv2.VideoCapture(self.video_path)

    #Functions for Calculating Each Optical Flow Methods
    def calculate_optical_flow_LK(self, prev_gray, gray, prev_pts):
        new_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_pts, None, winSize=(25, 25), maxLevel=3,
                                                        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))
        flow_image = np.zeros_like(prev_gray)
        for new, old in zip(new_pts, prev_pts):
            a, b = new.ravel()
            c, d = old.ravel()
            velocity_x = a - c
            velocity_y = b - d
            intensity = int(np.sqrt(velocity_x**2 + velocity_y**2) * 255 / 5)
            color = (intensity, 0, 0)
            flow_image = cv2.line(flow_image, (int(c), int(d)), (int(a), int(b)), color, 2)
        return flow_image

    def calculate_optical_flow_FB(self, prev_gray, gray):
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

        hsv = np.zeros_like(gray)  
        hsv[..., 1] = 255
        hsv[..., 0] = (angle * 180 / np.pi / 2).astype(np.uint8)
        hsv[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

        flow_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

        return flow_image

    #This one uses Shi-Tomasi for Lucas-Kanade
    def detect_corners(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners = cv2.goodFeaturesToTrack(gray, maxCorners=3000, qualityLevel=0.001, minDistance=0.1)
        return corners, gray
    

    #Tracking functions for Lucas-Kanade and Farneback algorithm
    def track_features_LK(self):
        self.get_video_path
        if not self.cap:
            return

        ret, frame = self.cap.read()
        prev_corners, prev_gray = self.detect_corners(frame)

        while ret:
            ret, frame = self.cap.read()
            if not ret:
                break

            new_corners, gray = self.detect_corners(frame)
            flow_image = self.calculate_optical_flow_LK(prev_gray, gray, prev_corners)

            prev_gray = gray
            prev_corners = new_corners.reshape(-1, 1, 2)

            cv2.imshow('Optical Flow', flow_image)
            k = cv2.waitKey(30) & 0xff
            if k == 27 or k == ord('q'):  # 'q' key to quit
                break

        self.cap.release()
        cv2.destroyAllWindows()

    def track_features_FB(self):
        while True:
            ret, frame = self.cap.read()
            if not ret:
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            flow_image = self.calculate_optical_flow_LK(self.prev_gray, gray)
            cv2.imshow('Optical Flow', flow_image)

            k = cv2.waitKey(30) & 0xff
            if k == 27 or k == ord('q'):  # 'q' key to quit
                break

            self.prev_gray = gray
            self.frame_count += 1

        self.cap.release()
        cv2.destroyAllWindows()


    #Functions for getting Optical Images
    def get_optical_images_LK(self, progress_var):
        self.get_video_path()
        if not self.cap:
            return

        progress_window = tk.Toplevel()  
        progress_window.title("Optical Flow Conversion Progress")
        screen_width = progress_window.winfo_screenwidth()
        screen_height = progress_window.winfo_screenheight()
        x_coordinate = (screen_width - 500) // 2
        y_coordinate = (screen_height - 500) // 2
        progress_window.geometry(f"500x200+{x_coordinate}+{y_coordinate}")

        progress_label = tk.Label(progress_window, text="Optical Flow Conversion Progress:")
        progress_label.pack(pady=10)

        progress_bar = ttk.Progressbar(progress_window, variable=progress_var, length=200, mode='determinate')
        progress_bar.pack(pady=10)

        if not self.cap:
            return

        self.create_output_folder()

        total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))

        ret, frame = self.cap.read()
        prev_corners, prev_gray = self.detect_corners(frame)

        while ret:
            ret, frame = self.cap.read()
            if not ret:
                break

            new_corners, gray = self.detect_corners(frame)
            flow_image = self.calculate_optical_flow_LK(prev_gray, gray, prev_corners)

            cv2.imwrite(os.path.join(self.output_folder, f'optical_flow_{self.frame_count + 1:04d}.png'), flow_image)

            progress_var.set(int((self.frame_count / total_frames) * 100))  # Update progress

            k = cv2.waitKey(30) & 0xff
            if k == 27 or k == ord('q'):  # 'q' key to quit
                break

            self.frame_count += 1
            prev_gray = gray
            prev_corners = new_corners.reshape(-1, 1, 2)

            progress_window.update()

        self.frame_count = 0
        self.cap.release()
        cv2.destroyAllWindows()

        progress_window.destroy()

    def get_optical_images_FB(self,progress_var):
        self.get_video_path()
        if not self.cap:
            return

        progress_window = tk.Toplevel()  
        progress_window.title("Optical Flow Conversion Progress")
        screen_width = progress_window.winfo_screenwidth()
        screen_height = progress_window.winfo_screenheight()
        x_coordinate = (screen_width - 500) // 2
        y_coordinate = (screen_height - 500) // 2
        progress_window.geometry(f"500x200+{x_coordinate}+{y_coordinate}")

        progress_label = tk.Label(progress_window, text="Optical Flow Conversion Progress:")
        progress_label.pack(pady=10)

        progress_bar = ttk.Progressbar(progress_window, variable=progress_var, length=200, mode='determinate')
        progress_bar.pack(pady=10)

        if not self.cap:
            return

        self.create_output_folder()

        total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        while True:
            ret, frame = self.cap.read()
            if not ret:
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            flow_image = self.calculate_optical_flow_FB(self.prev_gray, gray)
            cv2.imshow('Optical Flow', flow_image)

            image_filename = f'optical_flow_{self.frame_count:04d}.png'
            cv2.imwrite(os.path.join(self.output_folder, image_filename), flow_image)

            progress_var.set(int((self.frame_count / total_frames) * 100))  # Update progress

            k = cv2.waitKey(30) & 0xff
            if k == 27 or k == ord('q'):  # 'q' key to quit
                break

            self.prev_gray = gray
            self.frame_count += 1

        self.cap.release()
        cv2.destroyAllWindows()

    #Functions for showing Optical Flow
    def show_optical_flow_video_LK(self):
        self.get_video_path()
        self.track_features_LK()

    def show_optical_flow_video_FB(self):
        self.get_video_path()
        self.track_features_FB()

    def live_optical_flow_tracking(self):
        self.cap = cv2.VideoCapture(0)  # Use camera feed
        self.track_features()

class MainMenu:
    def __init__(self, root, algorithm_name, optical_flows):
        self.root = root
        self.root.title(f"{algorithm_name} Optical Flow Menu")

        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()
        x_coordinate = (screen_width - 500) // 2
        y_coordinate = (screen_height - 500) // 2
        self.root.geometry(f"500x200+{x_coordinate}+{y_coordinate}")
        self.optical_flows = optical_flows

        # Buttons
        btn_get_optical_images = tk.Button(root, text="Get Optical Images", command=self.get_optical_images)
        btn_get_optical_images.pack(pady=10)

        btn_show_optical_flow = tk.Button(root, text="Show Optical Flow Video", command=self.show_optical_flow_video)
        btn_show_optical_flow.pack(pady=10)

        btn_live_tracking = tk.Button(root, text="Live Optical Flow Tracking", command=self.live_optical_flow_tracking)
        btn_live_tracking.pack(pady=10)

        btn_back = tk.Button(root, text="Back", command=self.back_to_algorithm_selection)
        btn_back.pack(pady=10)

        btn_exit = tk.Button(root, text="Exit", command=self.exit_application)
        btn_exit.pack(pady=10)

    def get_optical_images(self):
        progress_var = tk.IntVar()
        progress_var.set(0)
        if "Lucas-Kanade" in self.root.title():
            self.optical_flows.get_optical_images_LK(progress_var)
        else:
            self.optical_flows.get_optical_images_FB(progress_var)

    def show_optical_flow_video(self):
        if "Lucas-Kanade" in self.root.title():
            self.optical_flows.show_optical_flow_video_LK()
        else:
            self.optical_flows.show_optical_flow_video_FB()

    def live_optical_flow_tracking(self):
        if "Lucas-Kanade" in self.root.title():
            self.optical_flows.live_optical_flow_tracking_LK()
        else:
            self.optical_flows.live_optical_flow_tracking_FB()

    def back_to_algorithm_selection(self):
        self.root.destroy()
        choose_algorithm_window()

    def exit_application(self):
        self.root.destroy()


def choose_algorithm_window():
    choose_algorithm_root = tk.Tk()
    choose_algorithm_root.title("Choose Optical Flow Algorithm")

    screen_width = choose_algorithm_root.winfo_screenwidth()
    screen_height = choose_algorithm_root.winfo_screenheight()
    x_coordinate = (screen_width - 500) // 2
    y_coordinate = (screen_height - 200) // 2
    choose_algorithm_root.geometry(f"500x200+{x_coordinate}+{y_coordinate}")

    lbl_choose_algorithm = tk.Label(choose_algorithm_root, text="Choose Optical Flow Algorithm:")
    lbl_choose_algorithm.pack(pady=10)

    def open_lucas_kanade_menu():
        choose_algorithm_root.destroy()
        root = tk.Tk()
        optical_flows = OpticalFlows()
        MainMenu(root, "Lucas-Kanade", optical_flows)

    def open_farneback_menu():
        choose_algorithm_root.destroy()
        root = tk.Tk()
        optical_flows = OpticalFlows()
        MainMenu(root, "Farneback", optical_flows)

    btn_lucas_kanade = tk.Button(choose_algorithm_root, text="Lucas-Kanade", command=open_lucas_kanade_menu)
    btn_lucas_kanade.pack(pady=10)

    btn_farneback = tk.Button(choose_algorithm_root, text="Farneback", command=open_farneback_menu)
    btn_farneback.pack(pady=10)

    choose_algorithm_root.mainloop()


if __name__ == "__main__":
    choose_algorithm_window()


Supine Neck Lift Optical Images Converter

In [1]:
import cv2
import numpy as np
import os
import tkinter as tk
from tkinter import filedialog, ttk

class LucasKanade_OpticalFlow:
    def __init__(self, video_path=None):
        self.video_path = video_path
        self.cap = cv2.VideoCapture(self.video_path) if self.video_path else None
        self.output_folder = None
        self.frame_count = 0
        self.input_folder = None  # Added line

    def create_output_folder(self):
        folder_number = 1

        while True:
            current_folder = f'Images/SNL/SNL_video_{folder_number:03d}'
            if not os.path.exists(current_folder):
                os.makedirs(current_folder)
                self.output_folder = current_folder
                break
            else:
                folder_number += 1
    
    def create_output_folders(self):  # Added method
        folder_number = 1

        while True:
            current_folder = f'Images/SNL/SNL_video_{folder_number:03d}'
            if not os.path.exists(current_folder):
                os.makedirs(current_folder)
                self.output_folder = current_folder
                break
            else:
                folder_number += 1

    def auto_output_folders(self):  # Corrected method name
        folder_number = 1

        while True:
            current_folder = f'Images/SNL/SNL_video_{folder_number:03d}'
            if not os.path.exists(current_folder):
                os.makedirs(current_folder)
                self.output_folder = current_folder
                break
            else:
                folder_number += 1

        video_files = [f for f in os.listdir(self.input_folder) if f.endswith('.mp4') or f.endswith('.avi')]

        for video_file in video_files:
            video_path = os.path.join(self.input_folder, video_file)
            self.create_output_folders()  # Create a new output folder for each video

            self.cap = cv2.VideoCapture(video_path)
            total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))

            ret, frame = self.cap.read()
            prev_corners, prev_gray = self.detect_corners(frame)

            while ret:
                ret, frame = self.cap.read()
                if not ret:
                    break

                new_corners, gray = self.detect_corners(frame)
                flow_image = self.calculate_optical_flow(prev_gray, gray, prev_corners)

                cv2.imwrite(os.path.join(self.output_folder, f'optical_flow_{self.frame_count + 1:04d}.png'), flow_image)

                progress_var.set(int((self.frame_count / total_frames) * 100))  # Update progress

                k = cv2.waitKey(30) & 0xff
                if k == 27 or k == ord('q'):  # 'q' key to quit
                    break

                self.frame_count += 1
                prev_gray = gray
                prev_corners = new_corners.reshape(-1, 1, 2)

                progress_window.update()

            self.frame_count = 0
            self.cap.release()

        cv2.destroyAllWindows()
        progress_window.destroy()

    def detect_corners(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners = cv2.goodFeaturesToTrack(gray, maxCorners=3000, qualityLevel=0.001, minDistance=0.1)
        return corners, gray

    def calculate_optical_flow(self, prev_gray, gray, prev_pts):
        new_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_pts, None, winSize=(25, 25), maxLevel=3,
                                                        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))
        flow_image = np.zeros_like(prev_gray)
        for new, old in zip(new_pts, prev_pts):
            a, b = new.ravel()
            c, d = old.ravel()
            velocity_x = a - c
            velocity_y = b - d
            intensity = int(np.sqrt(velocity_x**2 + velocity_y**2) * 255 / 5)
            color = (intensity, 0, 0)
            flow_image = cv2.line(flow_image, (int(c), int(d)), (int(a), int(b)), color, 2)
        return flow_image

    def get_video_path(self):
        file_path = filedialog.askopenfilename(title="Select Video File", filetypes=[("Video files", "*.mp4;*.avi")])
        if file_path:
            self.video_path = file_path
            self.cap = cv2.VideoCapture(self.video_path)

    def select_input_folder(self):
        root = tk.Tk()
        root.withdraw()  # Hide the main window
        self.input_folder = filedialog.askdirectory(title="Select Video Folder")
        if not self.input_folder:
            print("No folder selected. Exiting.")
        else:
            print(f"Selected folder: {self.input_folder}")

    def get_optical_images(self, progress_var):
        self.get_video_path()
        if not self.cap:
            return
        #adjust tkinter window to center
        progress_window = tk.Toplevel()  
        progress_window.title("Optical Flow Conversion Progress")
        screen_width = progress_window.winfo_screenwidth()
        screen_height = progress_window.winfo_screenheight()
        x_coordinate = (screen_width - 500) // 2
        y_coordinate = (screen_height - 500) // 2
        progress_window.geometry(f"500x200+{x_coordinate}+{y_coordinate}")

        progress_label = tk.Label(progress_window, text="Optical Flow Conversion Progress:")
        progress_label.pack(pady=10)

        progress_bar = ttk.Progressbar(progress_window, variable=progress_var, length=200, mode='determinate')
        progress_bar.pack(pady=10)

        if not self.cap:
            return

        self.create_output_folder()

        total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))

        ret, frame = self.cap.read()
        prev_corners, prev_gray = self.detect_corners(frame)

        while ret:
            ret, frame = self.cap.read()
            if not ret:
                break

            new_corners, gray = self.detect_corners(frame)
            flow_image = self.calculate_optical_flow(prev_gray, gray, prev_corners)

            cv2.imwrite(os.path.join(self.output_folder, f'optical_flow_{self.frame_count + 1:04d}.png'), flow_image)

            progress_var.set(int((self.frame_count / total_frames) * 100))  # Update progress

            k = cv2.waitKey(30) & 0xff
            if k == 27 or k == ord('q'):  # 'q' key to quit
                break

            self.frame_count += 1
            prev_gray = gray
            prev_corners = new_corners.reshape(-1, 1, 2)

            progress_window.update()

        self.frame_count = 0
        self.cap.release()
        cv2.destroyAllWindows()

        progress_window.destroy()

    def automate_optical_images(self, progress_var):
        self.select_input_folder()
        if not self.input_folder:
            return

        video_files = [f for f in os.listdir(self.input_folder) if f.endswith('.mp4') or f.endswith('.avi')]

        for video_file in video_files:
            video_path = os.path.join(self.input_folder, video_file)
            self.create_output_folders()  # Create a new output folder for each video

            self.cap = cv2.VideoCapture(video_path)
            total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))

            ret, frame = self.cap.read()
            prev_corners, prev_gray = self.detect_corners(frame)

            while ret:
                ret, frame = self.cap.read()
                if not ret:
                    break

                new_corners, gray = self.detect_corners(frame)
                flow_image = self.calculate_optical_flow(prev_gray, gray, prev_corners)

                cv2.imwrite(os.path.join(self.output_folder, f'optical_flow_{self.frame_count + 1:04d}.png'), flow_image)

                progress_var.set(int((self.frame_count / total_frames) * 100))  # Update progress

                k = cv2.waitKey(30) & 0xff
                if k == 27 or k == ord('q'):  # 'q' key to quit
                    break

                self.frame_count += 1
                prev_gray = gray
                prev_corners = new_corners.reshape(-1, 1, 2)

            self.frame_count = 0
            self.cap.release()

        cv2.destroyAllWindows()



    def show_optical_flow_video(self):
            self.get_video_path()
            self.track_features()

    def live_optical_flow_tracking(self):
        self.cap = cv2.VideoCapture(0)  # Use camera feed
        self.track_features()

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Lucas-Kanade Optical Flow")

    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    x_coordinate = (screen_width - 500) // 2
    y_coordinate = (screen_height - 500) // 2
    root.geometry(f"500x200+{x_coordinate}+{y_coordinate}")
    feature_tracker = LucasKanade_OpticalFlow()

    def get_optical_images():
        progress_var = tk.IntVar()
        progress_var.set(0)
        feature_tracker.get_optical_images(progress_var)
    
    def automate_optical_images():
        progress_var = tk.IntVar()
        progress_var.set(0)
        feature_tracker.automate_optical_images(progress_var)

    def show_optical_flow_video():
        feature_tracker.show_optical_flow_video()

    def live_optical_flow_tracking():
        feature_tracker.live_optical_flow_tracking()

    def exit_application():
        root.destroy()

    btn_get_optical_images = tk.Button(root, text="Get Optical Images", command=get_optical_images)
    btn_get_optical_images.pack(pady=10)

    btn_automate_optical_images = tk.Button(root, text="Automate Optical Images", command=automate_optical_images)
    btn_automate_optical_images.pack(pady=10)

    btn_show_optical_flow = tk.Button(root, text="Show Optical Flow Video", command=show_optical_flow_video)
    btn_show_optical_flow.pack(pady=10)

    btn_live_tracking = tk.Button(root, text="Live Optical Flow Tracking", command=live_optical_flow_tracking)
    btn_live_tracking.pack(pady=10)

    btn_exit = tk.Button(root, text="Exit", command=exit_application)
    btn_exit.pack(pady=10)

    root.mainloop()


Selected folder: G:/THESIS/Code/ComputerVision_Physiotherapy/SupineNeckLift


KeyboardInterrupt: 

Quadruped Thoracic Rotation Optical Images Converter 

In [10]:
import cv2
import numpy as np
import os
import tkinter as tk
from tkinter import filedialog, ttk

class LucasKanade_OpticalFlow:
    def __init__(self, video_path=None):
        self.video_path = video_path
        self.cap = cv2.VideoCapture(self.video_path) if self.video_path else None
        self.output_folder = None
        self.frame_count = 0
        self.input_folder = None  # Added line

    def create_output_folder(self):
        folder_number = 1

        while True:
            current_folder = f'Images/QTR/QTR_video_{folder_number:03d}'
            if not os.path.exists(current_folder):
                os.makedirs(current_folder)
                self.output_folder = current_folder
                break
            else:
                folder_number += 1
    
    def create_output_folders(self):  # Added method
        folder_number = 1

        while True:
            current_folder = f'Images/QTR/QTR_video_{folder_number:03d}'
            if not os.path.exists(current_folder):
                os.makedirs(current_folder)
                self.output_folder = current_folder
                break
            else:
                folder_number += 1

    def auto_output_folders(self):  # Corrected method name
        folder_number = 1

        while True:
            current_folder = f'Images/SNL/SNL_video_{folder_number:03d}'
            if not os.path.exists(current_folder):
                os.makedirs(current_folder)
                self.output_folder = current_folder
                break
            else:
                folder_number += 1

        video_files = [f for f in os.listdir(self.input_folder) if f.endswith('.mp4') or f.endswith('.avi')]

        for video_file in video_files:
            video_path = os.path.join(self.input_folder, video_file)
            self.create_output_folders()  # Create a new output folder for each video

            self.cap = cv2.VideoCapture(video_path)
            total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))

            ret, frame = self.cap.read()
            prev_corners, prev_gray = self.detect_corners(frame)

            while ret:
                ret, frame = self.cap.read()
                if not ret:
                    break

                new_corners, gray = self.detect_corners(frame)
                flow_image = self.calculate_optical_flow(prev_gray, gray, prev_corners)

                cv2.imwrite(os.path.join(self.output_folder, f'optical_flow_{self.frame_count + 1:04d}.png'), flow_image)

                progress_var.set(int((self.frame_count / total_frames) * 100))  # Update progress

                k = cv2.waitKey(30) & 0xff
                if k == 27 or k == ord('q'):  # 'q' key to quit
                    break

                self.frame_count += 1
                prev_gray = gray
                prev_corners = new_corners.reshape(-1, 1, 2)

                progress_window.update()

            self.frame_count = 0
            self.cap.release()

        cv2.destroyAllWindows()
        progress_window.destroy()

    def detect_corners(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners = cv2.goodFeaturesToTrack(gray, maxCorners=3000, qualityLevel=0.001, minDistance=0.1)
        return corners, gray

    def calculate_optical_flow(self, prev_gray, gray, prev_pts):
        new_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_pts, None, winSize=(25, 25), maxLevel=3,
                                                        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))
        flow_image = np.zeros_like(prev_gray)
        for new, old in zip(new_pts, prev_pts):
            a, b = new.ravel()
            c, d = old.ravel()
            velocity_x = a - c
            velocity_y = b - d
            intensity = int(np.sqrt(velocity_x**2 + velocity_y**2) * 255 / 5)
            color = (intensity, 0, 0)
            flow_image = cv2.line(flow_image, (int(c), int(d)), (int(a), int(b)), color, 2)
        return flow_image

    def get_video_path(self):
        file_path = filedialog.askopenfilename(title="Select Video File", filetypes=[("Video files", "*.mp4;*.avi")])
        if file_path:
            self.video_path = file_path
            self.cap = cv2.VideoCapture(self.video_path)

    def select_input_folder(self):
        root = tk.Tk()
        root.withdraw()  # Hide the main window
        self.input_folder = filedialog.askdirectory(title="Select Video Folder")
        if not self.input_folder:
            print("No folder selected. Exiting.")
        else:
            print(f"Selected folder: {self.input_folder}")

    def get_optical_images(self, progress_var):
        self.get_video_path()
        if not self.cap:
            return
        #adjust tkinter window to center
        progress_window = tk.Toplevel()  
        progress_window.title("Optical Flow Conversion Progress")
        screen_width = progress_window.winfo_screenwidth()
        screen_height = progress_window.winfo_screenheight()
        x_coordinate = (screen_width - 500) // 2
        y_coordinate = (screen_height - 500) // 2
        progress_window.geometry(f"500x200+{x_coordinate}+{y_coordinate}")

        progress_label = tk.Label(progress_window, text="Optical Flow Conversion Progress:")
        progress_label.pack(pady=10)

        progress_bar = ttk.Progressbar(progress_window, variable=progress_var, length=200, mode='determinate')
        progress_bar.pack(pady=10)

        if not self.cap:
            return

        self.create_output_folder()

        total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))

        ret, frame = self.cap.read()
        prev_corners, prev_gray = self.detect_corners(frame)

        while ret:
            ret, frame = self.cap.read()
            if not ret:
                break

            new_corners, gray = self.detect_corners(frame)
            flow_image = self.calculate_optical_flow(prev_gray, gray, prev_corners)

            cv2.imwrite(os.path.join(self.output_folder, f'optical_flow_{self.frame_count + 1:04d}.png'), flow_image)

            progress_var.set(int((self.frame_count / total_frames) * 100))  # Update progress

            k = cv2.waitKey(30) & 0xff
            if k == 27 or k == ord('q'):  # 'q' key to quit
                break

            self.frame_count += 1
            prev_gray = gray
            prev_corners = new_corners.reshape(-1, 1, 2)

            progress_window.update()

        self.frame_count = 0
        self.cap.release()
        cv2.destroyAllWindows()

        progress_window.destroy()

    def automate_optical_images(self, progress_var):
        self.select_input_folder()
        if not self.input_folder:
            return

        video_files = [f for f in os.listdir(self.input_folder) if f.endswith('.mp4') or f.endswith('.avi')]

        for video_file in video_files:
            video_path = os.path.join(self.input_folder, video_file)
            self.create_output_folders()  # Create a new output folder for each video

            self.cap = cv2.VideoCapture(video_path)
            total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))

            ret, frame = self.cap.read()
            prev_corners, prev_gray = self.detect_corners(frame)

            while ret:
                ret, frame = self.cap.read()
                if not ret:
                    break

                new_corners, gray = self.detect_corners(frame)
                flow_image = self.calculate_optical_flow(prev_gray, gray, prev_corners)

                cv2.imwrite(os.path.join(self.output_folder, f'optical_flow_{self.frame_count + 1:04d}.png'), flow_image)

                progress_var.set(int((self.frame_count / total_frames) * 100))  # Update progress

                k = cv2.waitKey(30) & 0xff
                if k == 27 or k == ord('q'):  # 'q' key to quit
                    break

                self.frame_count += 1
                prev_gray = gray
                prev_corners = new_corners.reshape(-1, 1, 2)

            self.frame_count = 0
            self.cap.release()

        cv2.destroyAllWindows()

    def show_optical_flow_video(self):
            self.get_video_path()
            self.track_features()

    def live_optical_flow_tracking(self):
        self.cap = cv2.VideoCapture(0)  # Use camera feed
        self.track_features()

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Lucas-Kanade Optical Flow")

    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    x_coordinate = (screen_width - 500) // 2
    y_coordinate = (screen_height - 500) // 2
    root.geometry(f"500x200+{x_coordinate}+{y_coordinate}")
    feature_tracker = LucasKanade_OpticalFlow()

    def get_optical_images():
        progress_var = tk.IntVar()
        progress_var.set(0)
        feature_tracker.get_optical_images(progress_var)
    
    def automate_optical_images():
        progress_var = tk.IntVar()
        progress_var.set(0)
        feature_tracker.automate_optical_images(progress_var)

    def show_optical_flow_video():
        feature_tracker.show_optical_flow_video()

    def live_optical_flow_tracking():
        feature_tracker.live_optical_flow_tracking()

    def exit_application():
        root.destroy()

    btn_get_optical_images = tk.Button(root, text="Get Optical Images", command=get_optical_images)
    btn_get_optical_images.pack(pady=10)

    btn_automate_optical_images = tk.Button(root, text="Automate Optical Images", command=automate_optical_images)
    btn_automate_optical_images.pack(pady=10)

    btn_show_optical_flow = tk.Button(root, text="Show Optical Flow Video", command=show_optical_flow_video)
    btn_show_optical_flow.pack(pady=10)

    btn_live_tracking = tk.Button(root, text="Live Optical Flow Tracking", command=live_optical_flow_tracking)
    btn_live_tracking.pack(pady=10)

    btn_exit = tk.Button(root, text="Exit", command=exit_application)
    btn_exit.pack(pady=10)

    root.mainloop()


Lumbar Side Bends Optical Images Converter

In [1]:
import cv2
import numpy as np
import os
import tkinter as tk
from tkinter import filedialog, ttk

class LucasKanade_OpticalFlow:
    def __init__(self, video_path=None):
        self.video_path = video_path
        self.cap = cv2.VideoCapture(self.video_path) if self.video_path else None
        self.output_folder = None
        self.frame_count = 0
        self.input_folder = None  # Added line

    def create_output_folder(self):
        folder_number = 1

        while True:
            current_folder = f'Images/LSB/LSB_video_{folder_number:03d}'
            if not os.path.exists(current_folder):
                os.makedirs(current_folder)
                self.output_folder = current_folder
                break
            else:
                folder_number += 1
    
    def create_output_folders(self):  # Added method
        folder_number = 1

        while True:
            current_folder = f'Images/LSB/LSB_video_{folder_number:03d}'
            if not os.path.exists(current_folder):
                os.makedirs(current_folder)
                self.output_folder = current_folder
                break
            else:
                folder_number += 1

    def auto_output_folders(self):  # Corrected method name
        folder_number = 1

        while True:
            current_folder = f'Images/LSB/LSB_video_{folder_number:03d}'
            if not os.path.exists(current_folder):
                os.makedirs(current_folder)
                self.output_folder = current_folder
                break
            else:
                folder_number += 1

        video_files = [f for f in os.listdir(self.input_folder) if f.endswith('.mp4') or f.endswith('.avi')]

        for video_file in video_files:
            video_path = os.path.join(self.input_folder, video_file)
            self.create_output_folders()  # Create a new output folder for each video

            self.cap = cv2.VideoCapture(video_path)
            total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))

            ret, frame = self.cap.read()
            prev_corners, prev_gray = self.detect_corners(frame)

            while ret:
                ret, frame = self.cap.read()
                if not ret:
                    break

                new_corners, gray = self.detect_corners(frame)
                flow_image = self.calculate_optical_flow(prev_gray, gray, prev_corners)

                cv2.imwrite(os.path.join(self.output_folder, f'optical_flow_{self.frame_count + 1:04d}.png'), flow_image)

                progress_var.set(int((self.frame_count / total_frames) * 100))  # Update progress

                k = cv2.waitKey(30) & 0xff
                if k == 27 or k == ord('q'):  # 'q' key to quit
                    break

                self.frame_count += 1
                prev_gray = gray
                prev_corners = new_corners.reshape(-1, 1, 2)

                progress_window.update()

            self.frame_count = 0
            self.cap.release()

        cv2.destroyAllWindows()
        progress_window.destroy()

    def detect_corners(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners = cv2.goodFeaturesToTrack(gray, maxCorners=3000, qualityLevel=0.001, minDistance=0.1)
        return corners, gray

    def calculate_optical_flow(self, prev_gray, gray, prev_pts):
        new_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_pts, None, winSize=(25, 25), maxLevel=3,
                                                        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))
        flow_image = np.zeros_like(prev_gray)
        for new, old in zip(new_pts, prev_pts):
            a, b = new.ravel()
            c, d = old.ravel()
            velocity_x = a - c
            velocity_y = b - d
            intensity = int(np.sqrt(velocity_x**2 + velocity_y**2) * 255 / 5)
            color = (intensity, 0, 0)
            flow_image = cv2.line(flow_image, (int(c), int(d)), (int(a), int(b)), color, 2)
        return flow_image

    def get_video_path(self):
        file_path = filedialog.askopenfilename(title="Select Video File", filetypes=[("Video files", "*.mp4;*.avi")])
        if file_path:
            self.video_path = file_path
            self.cap = cv2.VideoCapture(self.video_path)

    def select_input_folder(self):
        root = tk.Tk()
        root.withdraw()  # Hide the main window
        self.input_folder = filedialog.askdirectory(title="Select Video Folder")
        if not self.input_folder:
            print("No folder selected. Exiting.")
        else:
            print(f"Selected folder: {self.input_folder}")

    def get_optical_images(self, progress_var):
        self.get_video_path()
        if not self.cap:
            return
        #adjust tkinter window to center
        progress_window = tk.Toplevel()  
        progress_window.title("Optical Flow Conversion Progress")
        screen_width = progress_window.winfo_screenwidth()
        screen_height = progress_window.winfo_screenheight()
        x_coordinate = (screen_width - 500) // 2
        y_coordinate = (screen_height - 500) // 2
        progress_window.geometry(f"500x200+{x_coordinate}+{y_coordinate}")

        progress_label = tk.Label(progress_window, text="Optical Flow Conversion Progress:")
        progress_label.pack(pady=10)

        progress_bar = ttk.Progressbar(progress_window, variable=progress_var, length=200, mode='determinate')
        progress_bar.pack(pady=10)

        if not self.cap:
            return

        self.create_output_folder()

        total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))

        ret, frame = self.cap.read()
        prev_corners, prev_gray = self.detect_corners(frame)

        while ret:
            ret, frame = self.cap.read()
            if not ret:
                break

            new_corners, gray = self.detect_corners(frame)
            flow_image = self.calculate_optical_flow(prev_gray, gray, prev_corners)

            cv2.imwrite(os.path.join(self.output_folder, f'optical_flow_{self.frame_count + 1:04d}.png'), flow_image)

            progress_var.set(int((self.frame_count / total_frames) * 100))  # Update progress

            k = cv2.waitKey(30) & 0xff
            if k == 27 or k == ord('q'):  # 'q' key to quit
                break

            self.frame_count += 1
            prev_gray = gray
            prev_corners = new_corners.reshape(-1, 1, 2)

            progress_window.update()

        self.frame_count = 0
        self.cap.release()
        cv2.destroyAllWindows()

        progress_window.destroy()

    def automate_optical_images(self, progress_var):
        self.select_input_folder()
        if not self.input_folder:
            return

        video_files = [f for f in os.listdir(self.input_folder) if f.endswith('.mp4') or f.endswith('.avi')]

        for video_file in video_files:
            video_path = os.path.join(self.input_folder, video_file)
            self.create_output_folders()  # Create a new output folder for each video

            self.cap = cv2.VideoCapture(video_path)
            total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))

            ret, frame = self.cap.read()
            prev_corners, prev_gray = self.detect_corners(frame)

            while ret:
                ret, frame = self.cap.read()
                if not ret:
                    break

                new_corners, gray = self.detect_corners(frame)
                flow_image = self.calculate_optical_flow(prev_gray, gray, prev_corners)

                cv2.imwrite(os.path.join(self.output_folder, f'optical_flow_{self.frame_count + 1:04d}.png'), flow_image)

                progress_var.set(int((self.frame_count / total_frames) * 100))  # Update progress

                k = cv2.waitKey(30) & 0xff
                if k == 27 or k == ord('q'):  # 'q' key to quit
                    break

                self.frame_count += 1
                prev_gray = gray
                prev_corners = new_corners.reshape(-1, 1, 2)

            self.frame_count = 0
            self.cap.release()

        cv2.destroyAllWindows()



    def show_optical_flow_video(self):
            self.get_video_path()
            self.track_features()

    def live_optical_flow_tracking(self):
        self.cap = cv2.VideoCapture(0)  # Use camera feed
        self.track_features()

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Lucas-Kanade Optical Flow")

    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    x_coordinate = (screen_width - 500) // 2
    y_coordinate = (screen_height - 500) // 2
    root.geometry(f"500x200+{x_coordinate}+{y_coordinate}")
    feature_tracker = LucasKanade_OpticalFlow()

    def get_optical_images():
        progress_var = tk.IntVar()
        progress_var.set(0)
        feature_tracker.get_optical_images(progress_var)
    
    def automate_optical_images():
        progress_var = tk.IntVar()
        progress_var.set(0)
        feature_tracker.automate_optical_images(progress_var)

    def show_optical_flow_video():
        feature_tracker.show_optical_flow_video()

    def live_optical_flow_tracking():
        feature_tracker.live_optical_flow_tracking()

    def exit_application():
        root.destroy()

    btn_get_optical_images = tk.Button(root, text="Get Optical Images", command=get_optical_images)
    btn_get_optical_images.pack(pady=10)

    btn_automate_optical_images = tk.Button(root, text="Automate Optical Images", command=automate_optical_images)
    btn_automate_optical_images.pack(pady=10)

    btn_show_optical_flow = tk.Button(root, text="Show Optical Flow Video", command=show_optical_flow_video)
    btn_show_optical_flow.pack(pady=10)

    btn_live_tracking = tk.Button(root, text="Live Optical Flow Tracking", command=live_optical_flow_tracking)
    btn_live_tracking.pack(pady=10)

    btn_exit = tk.Button(root, text="Exit", command=exit_application)
    btn_exit.pack(pady=10)

    root.mainloop()
